In [1]:
from src.model_training.datasets.experiments_sanitize.no_sanitization import DefinitionDataset as OrigDataset
from src.model_training.datasets.experiments_sanitize.mixed_sanitization import DefinitionDataset
from src.model_training.datasets.experiments_sanitize.complete_sanitization import DefinitionDataset as CleanedDataset
from src.model_training.datasets.experiments_sanitize.complete_sanitization import DefinitionTestSet as CleanedTestDataset
from src.model_training.training.experiments_adapters.double_seq_bn import DefinitionModel

from tqdm.auto import tqdm
import regex as re
import pandas as pd
import pandasgui

old_special_idx = []


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
set_train_unsanitized, set_val_unsanitized = OrigDataset.create_dataset(tokenizer=tokenizer, shuffle=True, seed=42)

In [7]:
for x in set_train_unsanitized:
    if "{{übertr.:}}" in x["context_sentence"]:
        print(x)
        break

In [2]:
tokenizer, _ = DefinitionModel.create_model()
set_test = CleanedTestDataset.create_dataset(tokenizer=tokenizer, shuffle=True, seed=42)

Generating test split: 0 examples [00:00, ? examples/s]

Flattening the indices:   0%|          | 0/35738 [00:00<?, ? examples/s]

Map:   0%|          | 0/35738 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35726 [00:00<?, ? examples/s]

0it [00:00, ?it/s]

Filter:   0%|          | 0/35722 [00:00<?, ? examples/s]

In [ ]:
tokenizer, _ = DefinitionModel.create_model()
set_train_unsanitized, set_val_unsanitized = OrigDataset.create_dataset(tokenizer=tokenizer, shuffle=True, seed=42)
set_train_sanitized, set_val_sanitized = CleanedDataset.create_dataset(tokenizer=tokenizer, shuffle=True, seed=42)

old_special_idx = []

In [7]:
set_test[4]

{'title': 'Freilichtbühne',
 'context_word': 'Freilichtbühne',
 'context_sentence': '"Sie steht auf der großen Freilichtbühne und schaut etwas ungläubig."',
 'gt': 'Bühne unter freiem Himmel',
 'input_ids': [313,
  78295,
  259,
  13009,
  1015,
  442,
  9063,
  278,
  20757,
  38066,
  316,
  126123,
  472,
  259,
  36499,
  270,
  259,
  14427,
  263,
  259,
  1485,
  183123,
  12670,
  2454,
  7702,
  856,
  398,
  76360,
  795,
  20757,
  38066,
  316,
  126123,
  291,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [364, 126123, 4880, 22724, 470, 105849, 1],
 'prompt': '"Sie steht auf der großen Freilichtbühne und schaut etwas ungläubig." Was ist die Definition von Freilichtbühne? '}

In [5]:
debug_me = []

def global_sanitize(input_text):
    input_text = re.sub(r"<small>\[[0-9]*\]<\/small>", "", input_text)
    input_text = re.sub(r"<sup>.*?<\/sup>", "", input_text)
    input_text = re.sub(r"\(\[http:.*?''Internet-Beleg''\)", "", input_text)
    input_text = re.sub(r"<!?--.*?-->", "", input_text)
    input_text = re.sub(r"<\/?span.*?>", "", input_text)
    input_text = re.sub(r"\u200b", "", input_text)
    input_text = re.sub(r"(\u200b|\u200e|\u2060|\u200d|\u200c)", "", input_text)
    
    def remove_tags(input_text, tag):
        return input_text.replace(f"<{tag}>", "").replace(f"</{tag}>", "")

    input_text = remove_tags(input_text, "small")
    input_text = remove_tags(input_text, "big")
    input_text = remove_tags(input_text, "sub")
    input_text = remove_tags(input_text, "code")
    input_text = remove_tags(input_text, "u")
    input_text = remove_tags(input_text, "s")
    input_text = remove_tags(input_text, "small cap")
    
    regex = re.compile(r'<(ref|REF).*?</(ref|REF)>')
    input_text = regex.sub('', input_text)
    regex = re.compile(r'<(ref|REF).*?(ref|REF)>')
    input_text = regex.sub('', input_text)
    regex = re.compile(r'<(ref|REF).*?/>')
    input_text = regex.sub('', input_text)
    regex = re.compile(r'<div.*?</div>')
    input_text = regex.sub('', input_text)

    input_text = re.sub(r'\[\[([^\|\]]+)\]\]', r'\1', input_text)
    
    replacements_k = {
        "ugs.": "umgangssprachlich",
        "refl.": "", # "reflexiv",
        "trans.": "", # "transitiv",
        "abw.": "abwertend",
        "übertr.": "übertragen",
        "va.": "vor allem",
        "sal.": "salopp",
        "allg.": "allgemein",
        "auch": "",
        "kein Plural": ""
    }
    
    replacements_old = {
        "{{trans.|:}}": "",
        "{{trans.|,}}": "",
        '{{gM}}': "",
        '{{übertr.|:}}': "übertragen:",
        '{{ugs.|,}}': "umgangssprachlich,",
        "{{ugs.|:}}": "umgangssprachlich:",
        "{{ugs.}}": "umgangssprachlich",
        '{{QS Bedeutungen}}': "",
        '{{scherzh.|:}}': "scherzhaft:",
        '{{scherzh.}}': "scherzhaft:",
        '{{geh.|:}}': "gehoben:",
        '{{intrans.|:}}': "",
        '{{refl.|:}}': "",
        '{{kPl.|:}}': "",
        '{{kPl.}}': "",
        '{{fachspr.}}': "fachsprachlich",
        '{{schweiz.|:}}': "schweizerisch:",
        '{{österr.}}': "östereichisch:",
        '{{veraltend|:}}': "veraltend:",
        '{{refl.|,}}': "",
        '{{trans.}}': "",
        '{{NNBSP}}': " ",
        '{{QS Bedeutungen|unbelegt|spr=de}}': "",
        '{{QS Bedeutungen|unbelegt}}': "",
        '{{intrans.|,}}': "",
        '{{va.}}': "vor allem.",
        '{{intrans.|;}}': "",
        '{{abw.|:}}': "abwertend:",
        '{{abw.}}': "abwertend",
        '{{va.|:}}': "vor allem:",
        '{{trans.|;}}': "",
        '{{intrans.}}': "",
        '{{ugs.|}}': "umgangssprachlich",
        '{{vul.|:}}': "vulgär:",
        '{{QS Bedeutungen|stilistisch und semantisch seltsam}}': "",
        '{{refl.}}': "",
        '{{landsch.|:}}': "landschaftlich:",
        '{{geh.|,}}': "gehoben,",
        '{{QS Bedeutungen|keine Synonymauflistung, siehe Hilfe:Bedeutungen}}': "",
        '{{QS Bedeutungen|fehlend}}': "",
        '{{schweiz.|,}}': "schweizerisch,",
        '{{österr.|,}}': "östereichisch,",
        '{{österr.|:}}': "östereichisch:",
        '{{österr.|}}': "östereichisch",
        '{{QS Herkunft|fehlt}}': "",
        '₂': "2",
        '¹': "",
        '<!--erweitern-->': "",
        '²': "2",
        '³': "3",
        '⁴': "4",
        '×': "x",
        '−': "-",
        '_': "_",
        '′': "'",
        '½': "1/2",
        '‐': "-",
        '‑': "-",
        '̩': "",
        '±': "+-",
        '₃': "3",
        '″': "\"",
        '”': "\"",
        '`': "'",
        '‟': "\"",
        '⁻': "-",
        '₅': "5",
        '⁄': "/",
        '⅓': "1/3",
        '(¨)': "",
        '‛': "'",
        '©': "",
        '⁶': "6",
        '·': "x",
        "„": "\"", 
        "“": "\"", 
        "»": "\"", 
        "«": "\"", 
        "›": "\"", 
        "‹": "\"", 
        "‚": "\"", 
        "‘": "\"", 
        "´": "\"", 
        "’": "'", 
        "…": "...", 
        "–": "-", 
        "—": "-", 
        "®": "",
        "''": "",
        "<br>": " ",
        "</br>": " ",
        "<br/>": " ",
        "<br />": " ",
        "{{(R)}}": "",
        "&#91;sic&#93;": "[sic]",
        "&#8239;": " ",
        "&#x202f;": " ",
        "<nowiki>[</nowiki>": "",
        "<nowiki>]</nowiki>": "",
        "<!--": "",
        "□": "",
        "{{Neutr.}}": "",
        "\x93": "", 
        "\x96": "",
        "\x84": "",
        "\x80": "",
        '{{Beispiele fehlen}}': "",
        '{{CH&LI}}': "",
        '{{CURRENTYEAR}}': "",
        '{{Herkunft}}': "",
        '{{IPA}}': "",
        '{{Mask.}}': "",
        '{{Pl.}}': "",
        '{{es.}}': "",
        '{{fam.|:}}': "familiär:",
        '{{fam.|;}}': "familiär;",
        '{{gM|r}}': "",
        '{{hist.|:}}': "historisch",
        '{{kPl.|,}}': "",
        '{{kSt.}}': "",
        '{{landsch.|}}': "landschaftlich",
        '{{landsch.}}': "landschaftlich",
        '{{m}}': "m",
        '{{nordd.}}': "norddeutsch",
        '{{n}}': "n",
        '{{reg.}}': "",
        '{{scherzh.|,}}': "scherzhaft,",
        '{{schweiz.}}': "schweizerisch",
        '{{südd.|:}}': "süddeutsch:",
        '{{südd.|,}}': "süddeutsch,",
        '{{ugs.|;}}': "umgangssprachlich;",
        '{{va.|,}}': "vor allem,",
        '{{veraltet|:}}': "veraltet:",
        '{{vergleiche}}': "vergleiche",
        '{{übertr.:}}': "übertragen:",
        '{{übertr.}}': "übertragen",
        '{{übertr.|;}}': "übertragen;",
        "{{fig.|:}}": "figurativ:",
        '{{md.}}': "",
        '{{f}}': "",
        "→": "",
        "vatd.": "veraltend",
        "landsch.": "landschaftlich",
        "|sonst|": "|",
    }
    replacements = {
                '_': "_",
        '‐': "-",
        '‑': "-",
        '·': "x",
        '̩': "",
        '‛': "'",
        '”': "\"",
        '‟': "\"",
        '(¨)': "",
        'abw.': "abwertend",
        '{{Beispiele fehlen}}': "",
        'CH&LI': "",
        'CURRENTYEAR': "",
        '{{es.}}': "",
        '{{f}}': "",
        '{{fachspr.}}': "fachsprachlich",
        'fam.': "familiär;",
        '{{gM}}': "",
        '{{gM|r}}': "",
        '{{Herkunft}}': "",
        'hist.': "historisch",
        'intrans.': "",
        '{{IPA}}': "",
        'kPl.': "",
        'kSt.': "",
        'geh.': "gehoben",
        'landsch.': "landschaftlich",
        '{{m}}': "m",
        '{{Mask.}}': "",
        '{{md.}}': "",
        '{{n}}': "n",
        'NNBSP': " ",
        'nordd.': "norddeutsch",
        'österr.': "östereichisch:",
        'Pl.': "",
        'QS Bedeutungen': "",
        'QS Herkunft': "",
        'refl.': "",
        'reg.': "",
        'scherzh.': "scherzhaft",
        'schweiz.': "schweizerisch",
        'südd.': "süddeutsch,",
        'trans.': "",
        'übertr.': "übertragen:",
        'ugs.': "umgangssprachlich,",
        'va.': "vor allem.",
        '{{vergleiche}}': "vergleiche",
        'vul.': "vulgär:",
        '′': "'",
        '″': "\"",
        '`': "'",
        '©': "",
        '±': "+-",
        '×': "x",
        '<!--erweitern-->': "",
        '−': "-",
        '⁻': "-",
        '⁄': "/",
        '¹': "",
        '½': "1/2",
        '⅓': "1/3",
        '²': "2",
        '₂': "2",
        '³': "3",
        '₃': "3",
        '⁴': "4",
        '₅': "5",
        '⁶': "6",
        "–": "-", 
        "—": "-", 
        "…": "...", 
        "''": "",
        "’": "'", 
        " ́": "'",
        "‘": "\"", 
        "‚": "\"", 
        "‹": "\"", 
        "›": "\"", 
        "“": "\"", 
        "„": "\"", 
        "«": "\"", 
        "»": "\"", 
        "{{(R)}}": "",
        "fig.": "figurativ:",
        "{{Neutr.}}": "",
        "{{trans.|:}}": "",
        "\x80": "",
        "\x84": "",
        "\x93": "", 
        "\x96": "",
        "&#8239;": " ",
        "&#91;sic&#93;": "[sic]",
        "&#x202f;": " ",
        "´": "\"", 
        "®": "",
        "→": "",
        "<!--": "",
        "</br>": " ",
        "<br />": " ",
        "<br/>": " ",
        "<br>": " ",
        "<nowiki>[</nowiki>": "",
        "<nowiki>]</nowiki>": "",
        "|sonst|": "|",
        "□": "",
        "vatd.": "veraltend",
        "<nowiki/>": ""
    }
    
    def replace_k(k_text):
        k_text = k_text.replace("{{K|", "")
        k_text = k_text.replace("}}", "")
        k_text = k_text.split("|")
        use_elements = []
        for el in k_text:
            el = el.strip()
            if el in replacements_k:
                el = replacements_k[el]
            if "=" in el or "" == el:
                continue
            use_elements.append(el)
        if len(use_elements) > 0:
            return ", ".join(use_elements) + ":"
        else:
            return ""
    
    for key, value in replacements.items():
        input_text = input_text.replace(key, value)

    input_text = remove_tags(input_text, "nowiki")

    def keyword_last(input_text, keyword):
        return re.sub(r"{{" + keyword + "\|.*?}}", lambda u: u.group(0).replace("{{" + keyword + "|", "").replace("}}", "").split("|")[-1], input_text).strip()

    def keyword_delete(input_text, keyword):
        return re.sub(r"{{" + keyword + ".*?}}", "", input_text).strip()
    
    input_text = re.sub(r"\[\[Datei:.*?]]", "", input_text)

    # filter [[]] and [[|]]
    input_text = re.sub(r'\[\[([^\|\]]+)\|([^\|\]]+)\]\]', lambda u: u.group(2), input_text)

    # filter {{K|}}    
    input_text = re.sub(r"{{K\|.*?}}", lambda u: replace_k(u.group(0)), input_text).strip()

    # filter []
    input_text = input_text.replace("[...]", "!!REPLACEMENTPLACEHOLDER!!")
    input_text = re.sub(r'\[([^\|\]]+)\]', r'\1', input_text)
    input_text = input_text.replace("!!REPLACEMENTPLACEHOLDER!!", "[...]")
    # delete completely

    input_text = keyword_last(input_text, "Üt")
    input_text = keyword_last(input_text, "Ü")
    input_text = keyword_last(input_text, "Farbe")
    input_text = keyword_last(input_text, "L")

    input_text = keyword_delete(input_text, "Bibel")
    input_text = keyword_delete(input_text, "Anker")
    input_text = keyword_delete(input_text, "Audio")

    input_text = keyword_delete(input_text, "Internetquelle")
    input_text = keyword_last(input_text, "Polytonisch")
    input_text = keyword_delete(input_text, "QS Bedeutung(en)?")
    input_text = keyword_delete(input_text, "Literatur ")
    input_text = keyword_delete(input_text, "Literatur")

    input_text = keyword_delete(input_text, "Ref-dejure")
    input_text = keyword_delete(input_text, "Wikipedia")
    input_text = keyword_delete(input_text, "Wikisource")
    input_text = keyword_delete(input_text, "W\|")
    input_text = keyword_delete(input_text, "w\|")

    # take last entry only
    input_text = keyword_last(input_text, "Hintergrundfarbe")
    input_text = keyword_last(input_text, "Lautschrift")
    input_text = keyword_last(input_text, "WP")
    
    if "{{" in input_text:
        debug_me.append(input_text)
    
    input_text = re.sub(r'{{([^|]*?)[,;:]*?\|*?([:,;. ]*?)}}', r'\1\2', input_text)
    input_text = re.sub(r'{{[,;:]*?\|*?.*?}}', '', input_text)
    input_text = re.sub("{{}}", "", input_text)
    input_text = re.sub(".*?}}", "", input_text)
    input_text = input_text.replace(",:", ":")
    
    
    return re.sub("<ref>$", "", input_text.replace( "[[", "").replace("]]", "")).replace("-->", "").replace("<--", "").replace("<!--", "").replace("()", "").replace("ͤ", "").strip(", .:")



@classmethod
def _sanitize_context(cls, input_text: str) -> str:
    input_text = input_text.removesuffix("\n}}")
    input_text = cls._sanitize_spaces(input_text)
    

    # Substitute the matched characters with an empty string
    return global_sanitize(input_text)

@classmethod
def _sanitize_word(cls, input_text: str) -> str:
    input_text = input_text.removesuffix("\n}}")
    input_text = cls._sanitize_spaces(input_text)

    return global_sanitize(input_text)

@classmethod
def _sanitize_gt(cls, input_text: str) -> str:
    input_text = input_text.removesuffix("\n}}")
    input_text = cls._sanitize_spaces(input_text)

    return global_sanitize(input_text)

@classmethod
def _preprocessing(cls, examples):
    from src.prompting import prompt_pattern
    context_word = [cls._sanitize_word(w) for w in examples['context_word']]
    title = [cls._sanitize_word(w) for w in examples['title']]
    context_sentence = [cls._sanitize_context(w) for w in examples['context_sentence']]
    gt = [cls._sanitize_gt(w) for w in examples['gt']]
    input_texts = [prompt_pattern(c, w, pattern=cls.prompt_pattern) for c, w in zip(context_sentence, context_word)]
    inputs = cls.tokenizer(input_texts, max_length=512, truncation=True)
    inputs["labels"] = cls.tokenizer(text_target=gt, max_length=128, truncation=True)["input_ids"]
    inputs["prompt"] = input_texts # cls.tokenizer.batch_decode(inputs["input_ids"])
    inputs["title"] = title
    inputs["context_word"] = context_word
    inputs["context_sentence"] = context_sentence
    inputs["gt"] = gt # cls.tokenizer.batch_decode(inputs["labels"])

    return inputs


DefinitionDataset._sanitize_context = _sanitize_context
DefinitionDataset._sanitize_word = _sanitize_word
DefinitionDataset._sanitize_gt = _sanitize_gt
DefinitionDataset._preprocessing = _preprocessing

DefinitionDataset.tokenizer = tokenizer
dataset_train, dataset_val = DefinitionDataset._data_loading(True, 42, -1, -1)
set_train, set_val = DefinitionDataset._prepare_data(dataset_train, cache=False), DefinitionDataset._prepare_data(dataset_val, cache=False)

Map:   0%|          | 0/288148 [00:00<?, ? examples/s]

Map:   0%|          | 0/35664 [00:00<?, ? examples/s]

In [59]:
global_sanitize("{{trans.|:}} mit – insbesondere [[maschinell]] [[erzeugt]]em – [[Schnee]] [[bedecken]]")

'mit - insbesondere maschinell erzeugtem - Schnee bedecken'

In [7]:

print("RUNNING")
search_regex = r'[^\p{L}0-9 :"().,;!?\-/\'&%=$§@°\*€†≈~¥\+#]'
filter_regex = r'<math>'

def contains_special(input_string):
    input_string = input_string.replace("[...]", "!!REPLACEMENTPLACEHOLDER!!")
    regex = re.compile(search_regex)
    search_result = regex.search(input_string)
    # input_string.replace("!!REPLACEMENTPLACEHOLDER!!", "[...]")

    # Search the input string for any matches
    if search_result:
        return True
    return False

def contains_math(input_string):
    regex = re.compile(filter_regex)
    search_result = regex.search(input_string)

    # Search the input string for any matches
    if search_result:
        return True
    return False


special_idx = []
no_special_idx = []
new_no_special_idx = []

math_idx = []
no_math_idx = []

for x, datapoint in tqdm(enumerate(set_train)):
    if contains_special(datapoint['context_sentence']) or contains_special(datapoint['context_word']) or contains_special(datapoint['gt']):
        special_idx += [x]
        # print(f"{i['title']}: {i['context_word']} in {i['context_sentence']}\n{i['gt']}")
    else:
        no_special_idx += [x]
        if x in old_special_idx:
            new_no_special_idx += [x]

    if contains_math(datapoint['context_sentence']) or contains_math(datapoint['context_word']) or contains_math(datapoint['gt']):
        math_idx += [x]
        # print(f"{i['title']}: {i['context_word']} in {i['context_sentence']}\n{i['gt']}")
    else:
        no_math_idx += [x]

print(len(special_idx))
print(len(math_idx))
print(len(no_special_idx))
old_special_idx = list(special_idx)
df = pd.DataFrame.from_dict(set_train)
df.drop(columns=['input_ids', 'attention_mask', 'labels'], inplace=True)
special = df.iloc[special_idx].drop(index=math_idx)
no_special = df.drop(index=set(special_idx + math_idx))
new_no_special = df.iloc[new_no_special_idx]

# TODO REMOVE MATH FROM DATASET!

math = df.iloc[math_idx]
no_math = df.drop(index=math_idx)

pandasgui.show(special, no_special, new_no_special, math, no_math)

RUNNING


0it [00:00, ?it/s]

469
0
287553


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
/tmp/ipykernel_909483/1914664112.py:63: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_909483/1914664112.py:63: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_909483/1914664112.py:63: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_909483/1914664112.py:63: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a futur

In [23]:
DefinitionDataset._sanitize_spaces(dataset_train[71905]['context_sentence'])

"„Basis für ''Sonnensegel'' ist der Strahlungsdruck des Lichts. Die Teilchen die unser ''Sonnensegel'' als Antrieb nutzt sind Lichtteilchen (Photonen).“<ref>[http://www.bernd-leitenberger.de/sonnensegel.shtml Sonnensegel]''www.bernd-leitenberger.de'', abgerufen am 15. Juli 2013</ref> [[Datei:Bat_roof.jpeg|thumb|[3] Das Dach mit Solarmodulen als ''Sonnensegel'']]"

In [10]:
problematic_els = []

for i, el in enumerate(set_train):
    if ".:" in el['gt'] or ".," in el['gt'] or ".;" in el['gt']:
        problematic_els += [i]

In [11]:
len(problematic_els)

2506

In [12]:
df = pd.DataFrame.from_dict(set_train)
df.drop(columns=['input_ids', 'attention_mask', 'labels'], inplace=True)
pandasgui.show(df.iloc[problematic_els])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_1336890/485035752.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pandasgui.show(df.iloc[problematic_els])
/tmp/ipykernel_1336890/485035752.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pandasgui.show(df.iloc[problematic_els

In [67]:
idx_it = iter(range(len(special)))

In [146]:
def search_special(input_string):
    input_string = input_string.replace("[...]", "!!REPLACEMENTPLACEHOLDER!!")
    regex = re.compile(search_regex)
    search_result = regex.search(input_string)
    input_string.replace("!!REPLACEMENTPLACEHOLDER!!", "[...]")

    # Search the input string for any matches
    return search_result

idx = next(idx_it)
print(idx)
print(search_special(special.iloc[idx]['context_sentence']))
print(search_special(special.iloc[idx]['gt']))

1
None
<regex.Match object; span=(193, 251), match='{{QS Bedeutungen|beschreibt ein Substantiv kein Adjektiv}}'>


In [72]:
search_special(special.iloc[idx]['context_sentence'])
special.iloc[idx]['context_sentence']

'"Wenn der Landes\u200bwolfsbeauftragte ihn als Problemwolf einstuft, kann er geschossen werden."'

In [46]:
found_matches = []
relevant_sentences = []
for _, i in special.iterrows():
    regex = re.compile(search_regex)
    if a := regex.search(i['context_sentence']):
        found_matches.append(a.group(0))
        relevant_sentences.append(i['context_sentence'])
    if a := regex.search(i['gt']):
        found_matches.append(a.group(0))
        relevant_sentences.append(i['gt'])

In [47]:
set(found_matches)

set()

In [48]:
from collections import Counter

Counter(found_matches)

Counter()

In [34]:
pandasgui.show(relevant_sentences)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [ ]:
"(\u200b|\u200e|\u2060|\u200d|\u200c)"|\x96': 1,
         '\x80': 1,
         '\x93': 1,
         '\xad': 1,
         '\ufeff': 1,
         '\x84': 1,}

In [43]:
text = "This is Test <!--, ''auch:'' [[erste]]/[[erster|-r]]/[[erstes|-s]], [[beste]]/[[bester|-r]]/[[bestes|-s]], [[extremste]]/[[extremste|-r]]/[[extremste|-s]]--> Wow!!! <!-- Test -->"
cleaned_text = re.sub(r'<!--.*?-->', '', text)

print(cleaned_text)


This is Test  Wow!!! 
